In [1]:
from google.colab import drive
import os
drive.mount('/content/gdrive')
os.chdir("gdrive/MyDrive/Yin/landscape")

Mounted at /content/gdrive


In [2]:
import pandas as pd
import networkx as nx
import numpy as np
from ast import literal_eval
import matplotlib.pyplot as plt

In [3]:
def get_dict(name):
	# mFile = open("analysis/data/forest_landscape_restoration_country_tab.tsv", "r",encoding="cp437", errors='ignore')
	# mFile = open("analysis/data/landscape_restoration_country_tab.tsv", "r",encoding="cp437", errors='ignore')
	# mFile = open("analysis/data/ecosystem_restoration_country_tab.tsv", "r",encoding="cp437", errors='ignore')
	# mFile = open("analysis/data/ecological_restoration_country_tab.tsv", "r",encoding="cp437", errors='ignore')
	mFile = open("analysis/data/" + name + "_country_tab.tsv", "r",encoding="cp437", errors='ignore')
	next(mFile)
	countries = {}
	for ele in mFile:
		data = ele.rstrip().split('\t')
		country = data[-3]
		coord = data[-2]
		author = data[-33]
		if author in countries:
			pass
		else:
			countries[author] = [country,coord]
	return countries

In [ ]:
def set_node_community(G, communities):
  for c, v_c in enumerate(communities):
    for v in v_c:
      G.nodes[v]['community'] = c + 1
def set_edge_community(G):
  for v, w, in G.edges:
    if G.nodes[v]['community'] == G.nodes[w]['community']:
      G.edges[v, w]['community'] = G.nodes[v]['community']
    else:
      G.edges[v, w]['community'] = 0
def get_color(i, r_off=1, g_off=1, b_off=1):
  r0, g0, b0 = 0, 0, 0
  n = 16
  low, high = 0.1, 0.9
  span = high - low
  r = low + span * (((i + r_off) * 3) % n) / (n - 1)
  g = low + span * (((i + g_off) * 5) % n) / (n - 1)
  b = low + span * (((i + b_off) * 7) % n) / (n - 1)
  return (r, g, b)


In [ ]:
###This is the code that restrict nodes with country informaiton only
import networkx as nx
from matplotlib import pyplot as plt
plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams.update({'figure.figsize': (15, 10)})
# get reproducible results
import networkx.algorithms.community as nxcom

import random
from numpy import random as nprand
random.seed(123)
nprand.seed(123)

G = nx.Graph()

df = pd.read_csv("analysis/data/forest_landscape_restoration_country_tab.tsv", sep ='\t')
# df = pd.read_csv("analysis/data/landscape_restoration_country_tab.tsv", sep ='\t')
# df = pd.read_csv("analysis/data/ecosystem_restoration_country_tab.tsv", sep ='\t')
# df = pd.read_csv("analysis/data/ecological_restoration_country_tab.tsv", sep ='\t')

df['retweeted_username'].replace('', np.nan, inplace=True)
df.dropna(subset=['retweeted_username'], inplace=True)
df['entities.mentions'].replace('', np.nan, inplace=True)
df.dropna(subset=['entities.mentions'], inplace=True)
df['entities.mentions'] = df['entities.mentions'].apply(literal_eval)
# print(df.head())
countries = get_dict("forest_landscape_restoration")
for index, row in df.iterrows():
  orgNode = row["retweeted_username"]
  mcoord = row["coord"]
  mym = row["year_month"]
  myear = row["year"]
  if orgNode in countries:
    if orgNode in G.nodes():
      nx.set_node_attributes(G, { orgNode:{'coord': mcoord, 'ym':mym, 'year': myear, 'country':countries[orgNode]}})
    else:
      #'person1', name='John Doe', age=40
      G.add_node(orgNode, coord = mcoord, ym = mym, year = myear, country = countries[orgNode])
  else:
    pass
  data = row["entities.mentions"]
  for ele in data:
    destNode = str(ele).replace("@", "")
    if destNode in countries:
      if destNode in G.nodes():
        nx.set_node_attributes(G, { destNode:{'coord': countries[destNode][1], 'ym':mym, 'year': myear, 'country':countries[destNode]}})
        G.add_edge(orgNode, destNode)
      else:
        G.add_node(destNode, ym = mym, year = myear, country = countries[destNode], coord = countries[destNode][1])
        G.add_edge(orgNode,destNode)
# print(G.edges())
    # print(row["retweeted_username"], row["entities.mentions"])
remove = [node for node,degree in dict(G.degree()).items() if degree < 3]
G.remove_nodes_from(remove)
G.remove_edges_from(nx.selfloop_edges(G))
print(G.nodes())

# communities = sorted(nxcom.greedy_modularity_communities(G), key=len, reverse=True)
#     # Count the communities
# print(f"The graph has {len(communities)} communities.")
# for ele in communities:
#   print(ele)


<ipython-input-16-5e5679e78ae3>:18: DtypeWarning: Columns (29,31,32,33,39,54,55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("analysis/data/forest_landscape_restoration_country_tab.tsv", sep ='\t')


['ICRAF', 'FAOForestry', 'FAO', 'FAOBrussels', 'FAOclimate', 'UNBiodiversity', 'min_waterUg', 'IucnRwanda', 'IUCN', 'IUCN_forests', 'UNEP', 'theGEF', 'RwandaForestry', 'GPFLRtweets', 'atree_org', 'andyheald', 'NGP_TechAssist', 'CIFOR', 'FAOPakistan', 'suechomba', 'RegreenAfrica', 'moefcc', 'landscapelab_', 'forest_everest', 'GlobalLF', 'Afr100_Official', 'IUFRO', 'StateHouseKenya', 'FAOAfrica', 'ladaniela308', 'Yale_ELTI', 'EnviroRwanda', 'RwandaGov', 'YIL_Initiative', 'PlanetaryPraxis', 'WorldResources', 'restoreforward', 'YaleEnvironment', 'Yale', 'madasascientist', 'woodlandleague', 'RECOR_Rw', 'REMA_Rwanda', 'EnvironmentRw', 'GovernorMwadime', 'AWF_Official', 'artofresearch', 'DanielWiegant', 'TaitaTaveta006', 'JAppliedEcology', 'FAOMalawi', 'iki_germany', 'DavidKaimowitz', 'giz_gmbh', 'Loah_Org', 'WRIafrica', 'MDPIOpenAccess', 'BMUV', 'Environment_Ke', 'WWF_Kenya', 'WWFTANZANIA', 'BiovIntCIAT_eng', 'ChrisJKettle', 'CGIAR', 'FMNRHub', 'NatureBasedSols', 'forestservice', 'FAOinNepal

In [4]:
import networkx as nx
from matplotlib import pyplot as plt
plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams.update({'figure.figsize': (15, 10)})
# get reproducible results
import networkx.algorithms.community as nxcom

import random
from numpy import random as nprand
random.seed(123)
nprand.seed(123)

def get_nodes_attr(name):
  mFile1 = open("data/re/" + name + "_top1.txt", "w")
  mFile2 = open("data/re/" + name + "_top2.txt", "w")
  mFile3 = open("data/re/" + name + "_all_actors.txt", "w")
  ###This is the code that restrict nodes with country informaiton only
  G = nx.Graph()
  df = pd.read_csv("analysis/data/" + name + "_country_tab.tsv", sep ='\t')

  df['retweeted_username'].replace('', np.nan, inplace=True)
  df.dropna(subset=['retweeted_username'], inplace=True)
  df['entities.mentions'].replace('', np.nan, inplace=True)
  df.dropna(subset=['entities.mentions'], inplace=True)
  df['entities.mentions'] = df['entities.mentions'].apply(literal_eval)
  countries = get_dict(name)
  for index, row in df.iterrows():
    orgNode = row["retweeted_username"]
    mcoord = row["coord"]
    mym = row["year_month"]
    myear = row["year"]
    if orgNode in countries:
      if orgNode in G.nodes():
        nx.set_node_attributes(G, { orgNode:{'coord': mcoord, 'ym':mym, 'year': myear, 'country':countries[orgNode]}})
      else:
        #'person1', name='John Doe', age=40
        G.add_node(orgNode, coord = mcoord, ym = mym, year = myear, country = countries[orgNode])
    else:
      pass
    data = row["entities.mentions"]
    for ele in data:
      destNode = str(ele).replace("@", "")
      if destNode in countries:
        if destNode in G.nodes():
          nx.set_node_attributes(G, { destNode:{'coord': countries[destNode][1], 'ym':mym, 'year': myear, 'country':countries[destNode]}})
          G.add_edge(orgNode, destNode)
        else:
          G.add_node(destNode, ym = mym, year = myear, country = countries[destNode], coord = countries[destNode][1])
          G.add_edge(orgNode,destNode)
  remove = [node for node,degree in dict(G.degree()).items() if degree < 3]
  G.remove_nodes_from(remove)
  G.remove_edges_from(nx.selfloop_edges(G))
  # print(G.nodes())
  for ele in G.nodes():
    # print(ele)-
    mFile3.write(ele + "\n")
  mFile3.close()

  communities = sorted(nxcom.greedy_modularity_communities(G), key=len, reverse=True)
  print(f"The graph has {len(communities)} communities.")
  for ele in communities[0]:
    mFile1.write(ele + "\n")
  mFile1.close()
  for ele2 in communities[1]:
    mFile2.write(ele2 + "\n")
  mFile2.close()

In [5]:
get_nodes_attr("forest_landscape_restoration")
get_nodes_attr("landscape_restoration")
get_nodes_attr("ecosystem_restoration")
get_nodes_attr("ecological_restoration")

<ipython-input-4-7ebfaed70376>:19: DtypeWarning: Columns (29,31,32,33,39,54,55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("analysis/data/" + name + "_country_tab.tsv", sep ='\t')


The graph has 31 communities.


<ipython-input-4-7ebfaed70376>:19: DtypeWarning: Columns (29,31,32,33,39) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("analysis/data/" + name + "_country_tab.tsv", sep ='\t')


The graph has 89 communities.


<ipython-input-4-7ebfaed70376>:19: DtypeWarning: Columns (29,31,32,33,39,54,55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("analysis/data/" + name + "_country_tab.tsv", sep ='\t')


The graph has 166 communities.


<ipython-input-4-7ebfaed70376>:19: DtypeWarning: Columns (7,29,31,32,33,54,55,56,57,58,59,60,61,62,63,64) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("analysis/data/" + name + "_country_tab.tsv", sep ='\t')


The graph has 138 communities.


In [7]:
import sys
def top1():
  # FR =[]
  # LR =[]
  # EcosR =[]
  # EcolR =[]

  # m1 = open("data/re/forest_landscape_restoration_top1.txt","r")
  # for ele in m1:
  #   FR.append(ele.rstrip())

  # m2 = open("data/re/landscape_restoration_top1.txt","r")
  # for ele2 in m2:
  #   LR.append(ele2.rstrip())

  # m3 = open("data/re/ecosystem_restoration_top1.txt","r")
  # for ele3 in m3:
  #   EcosR.append(ele3.rstrip())

  # m4 = open("data/re/ecological_restoration_top1.txt","r")
  # for ele4 in m4:
  #   EcolR.append(ele4.rstrip())

  FR =[]
  LR =[]
  EcosR =[]
  EcolR =[]

  m1 = open("data/re/forest_landscape_restoration_top2.txt","r")
  for ele in m1:
    FR.append(ele.rstrip())

  m2 = open("data/re/landscape_restoration_top2.txt","r")
  for ele2 in m2:
    LR.append(ele2.rstrip())

  m3 = open("data/re/ecosystem_restoration_top2.txt","r")
  for ele3 in m3:
    EcosR.append(ele3.rstrip())

  m4 = open("data/re/ecological_restoration_top2.txt","r")
  for ele4 in m4:
    EcolR.append(ele4.rstrip())



  # p =[FR, LR, EcosR, EcolR]
  # p =[FR, LR, EcosR]
  # p =[FR, LR]

  # p =[FR, EcosR, EcolR]
  # p =[FR, EcolR]
  # p =[FR, EcosR]

  # p =[EcosR, EcolR]

  # p =[LR, EcosR, EcolR]
  # p =[LR, EcosR]
  # p =[LR, EcolR]

  p = [FR, LR, EcolR]

  common_elements = set.intersection(*[set(list) for list in p])

  print(common_elements)

# def top2():
# 	list1 =[]
# 	list2 =[]
# 	list3 =[]
# 	list4 =[]

# 	m1 = open("data/re/forest_landscape_restoration_top1.txt","r")
# 	for ele in m1:
# 		list1.append(ele.rstrip())

# 	m2 = open("data/re/forest_landscape_restoration_top1.txt","r")
# 	for ele2 in m2:
# 		list2.append(ele2.rstrip())

# 	m3 = open("data/re/forest_landscape_restoration_top1.txt","r")
# 	for ele3 in m3:
# 		list3.append(ele3.rstrip())

# 	m4 = open("data/re/forest_landscape_restoration_top1.txt","r")
# 	for ele4 in m4:
# 		list4.append(ele4.rstrip())

# 	p =[list1, list2, list3]

# 	common_elements = set.intersection(*[set(list) for list in p])

# 	print(common_elements)



if __name__ == '__main__':
	top1()
	# top2()

set()


In [8]:
import sys
def top1():
  FR =[]
  LR =[]
  EcosR =[]
  EcolR =[]

  m1 = open("data/re/forest_landscape_restoration_all_actors.txt","r")
  for ele in m1:
    FR.append(ele.rstrip())

  m2 = open("data/re/landscape_restoration_all_actors.txt","r")
  for ele2 in m2:
    LR.append(ele2.rstrip())

  m3 = open("data/re/ecosystem_restoration_all_actors.txt","r")
  for ele3 in m3:
    EcosR.append(ele3.rstrip())

  m4 = open("data/re/ecological_restoration_all_actors.txt","r")
  for ele4 in m4:
    EcolR.append(ele4.rstrip())



  # p =[FR, LR, EcosR, EcolR]
  # p =[FR, LR, EcosR]
  # p =[FR, LR]

  # p =[FR, EcosR, EcolR]
  # p =[FR, EcolR]
  # p =[FR, EcosR]

  # p =[EcosR, EcolR]

  # p =[LR, EcosR, EcolR]
  # p =[LR, EcosR]
  # p =[LR, EcolR]
  p = [FR, LR, EcolR]

  common_elements = set.intersection(*[set(list) for list in p])

  print(common_elements)



if __name__ == '__main__':
	top1()

{'GlobalLF', 'CGIAR', 'unredd', 'usfs_r5', 'ForestsMatter', 'StevenLawry', 'forestideas', 'Forest_Research', 'StephenWoroniec', 'UNEP', 'theGEF', 'CreandoRedesRE', 'FLAREglobal', 'WorldResources', 'WLE_CGIAR', 'ConservationOrg', 'elwyngj', 'IFPRI', 'CrowtherLab', 'forestservice', 'Connect4Climate', 'FTA_Partnership', 'UNBiodiversity', 'FAOForestry', 'YIL_Initiative', 'cookswelljikos', 'Environment_Ke', 'jmollins', 'PIB_India', 'PIM_CGIAR', 'atree_org', 'ISAbella_P_PARF', 'josemreybenayas', 'ERI_NAU', 'SERestoration', 'restoreforward', 'ICRAF', 'suechomba', 'IPBES', 'MDPIOpenAccess', 'kundan_kumarkun', 'RegreenAfrica', 'lifefluvial', 'ErikSolheim', 'ForestRightsAct', 'CGIARnutrition', 'Focali_se', 'theGCF', 'SundayGeofrey5', 'lucbas1', 'MumbaMusondam', 'UNCCD', 'partnersrcn', 'CIFOR_bosques', 'JAppliedEcology', 'TimChristo', 'IUCN', 'NEPAD_Agency', 'moefcc', 'CWatsonICRAF', 'andyheald', 'Afr100_Official', 'min_waterUg', 'Yale_ELTI', 'CIFOR', 'BiovIntCIAT_eng', 'TonyJuniper', 'SIANIAgri'

In [ ]:
# Set node and edge communities
set_node_community(G, communities)
set_edge_community(G)
node_color = [get_color(G.nodes[v]['community']) for v in G.nodes]
# Set community color for edges between members of the same community (internal) and intra-community edges (external)
# external = [(v, w) for v, w in G.edges if G.edges[v, w]['community'] == 0]
# internal = [(v, w) for v, w in G.edges if G.edges[v, w]['community'] > 0]

external = [(v, w) for v, w in G.edges if G.edges[v, w]['community'] > 6]
internal = [(v, w) for v, w in G.edges if G.edges[v, w]['community'] < 7]

internal_color = ['black' for e in internal]

In [21]:
import networkx as nx
from matplotlib import pyplot as plt
import networkx.algorithms.community as nxcom

import random
from numpy import random as nprand
random.seed(123)
nprand.seed(123)

def get_nodes_attr(name):

  mFile1 = open("data/tscan/" + name + "_graph_org.txt", "w")
  mFile2 = open("data/tscan/" + name + "_graph_dest.txt", "w")

  G = nx.Graph()
  df = pd.read_csv("analysis/data/" + name + "_country_tab.tsv", sep ='\t')

  df['retweeted_username'].replace('', np.nan, inplace=True)
  df.dropna(subset=['retweeted_username'], inplace=True)
  df['entities.mentions'].replace('', np.nan, inplace=True)
  df.dropna(subset=['entities.mentions'], inplace=True)
  df['entities.mentions'] = df['entities.mentions'].apply(literal_eval)
  countries = get_dict(name)
  for index, row in df.iterrows():
    orgNode = row["retweeted_username"]
    mcoord = row["coord"]
    mym = row["year_month"]
    myear = row["year"]
    # mString = ''
    if orgNode in countries:
      # if orgNode in G.nodes():
      mString = orgNode + "\t" + str(myear) + "\n"
      mFile1.write(mString)
      #   nx.set_node_attributes(G, {orgNode:{'coord': mcoord, 'ym':mym, 'year': myear, 'country':countries[orgNode]}})
      # else:
      #   #'person1', name='John Doe', age=40
      #   G.add_node(orgNode, coord = mcoord, ym = mym, year = myear, country = countries[orgNode])
    else:
      pass
    data = row["entities.mentions"]
    for ele in data:
      destNode = str(ele).replace("@", "")
      if destNode in countries:
        mString2 = destNode + "\t" + str(myear) + "\n"
        # print(mString)
        mFile2.write(mString2)
        # if destNode in G.nodes():
        #   nx.set_node_attributes(G, { destNode:{'coord': countries[destNode][1], 'ym':mym, 'year': myear, 'country':countries[destNode]}})
        #   G.add_edge(orgNode, destNode)

        # else:
        #   G.add_node(destNode, ym = mym, year = myear, country = countries[destNode], coord = countries[destNode][1])
        #   G.add_edge(orgNode,destNode)
  # remove = [node for node,degree in dict(G.degree()).items() if degree < 3]
  # G.remove_nodes_from(remove)
  # G.remove_edges_from(nx.selfloop_edges(G))
  # # print(G.nodes())
  # # for ele in G.nodes():
  # #   print(G.nodes[ele])
  # print(G.edges())
  mFile1.close()
  mFile2.close()

In [25]:
get_nodes_attr("forest_landscape_restoration")
get_nodes_attr("landscape_restoration")
get_nodes_attr("ecosystem_restoration")
get_nodes_attr("ecological_restoration")

<ipython-input-21-c0328af7192c>:16: DtypeWarning: Columns (29,31,32,33,39,54,55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("analysis/data/" + name + "_country_tab.tsv", sep ='\t')
<ipython-input-21-c0328af7192c>:16: DtypeWarning: Columns (29,31,32,33,39) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("analysis/data/" + name + "_country_tab.tsv", sep ='\t')
<ipython-input-21-c0328af7192c>:16: DtypeWarning: Columns (29,31,32,33,39,54,55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("analysis/data/" + name + "_country_tab.tsv", sep ='\t')
<ipython-input-21-c0328af7192c>:16: DtypeWarning: Columns (7,29,31,32,33,54,55,56,57,58,59,60,61,62,63,64) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("analysis/data/" + name + "_country_tab.tsv", sep ='\t')


In [32]:
import sys
def keyactors(name):
  table = {}
  mFile1 = open("data/tscan/" + name + "_graph_org.txt", "r")
  for ele in mFile1:
    data = ele.strip().split("\t")
    if data[1] in table:
      table[data[1]].append(data[0])
    else:
      table[data[1]] = []

  mFile2 = open("data/tscan/" + name + "_graph_dest.txt", "r")
  for ele2 in mFile2:
    data2 = ele2.strip().split("\t")
    if data2[1] in table:
      table[data2[1]].append(data2[0])
    else:
      table[data2[1]] = []

  p =[]
  for key, value in table.items():
    p.append(value)

  common_elements = set.intersection(*[set(list) for list in p])

  print(common_elements)


keyactors("forest_landscape_restoration")
keyactors("landscape_restoration")
keyactors("ecosystem_restoration")
keyactors("ecological_restoration")

{'IUCN_forests', 'CIFOR_hutan', 'FAOForestry', 'BiovIntCIAT_eng', 'USDA', 'CGIAR', 'GlobalLF', 'EnvironmentRw', 'WorldResources', 'IUCN', 'ICRAF', 'IUFRO'}
{'IUCN_forests', 'theGEF', 'EnvironmentRw', 'forestservice', 'WLE_CGIAR', 'CIFOR_hutan', 'WorldBank', 'SIANIAgri', 'TimChristo', 'WRIIndia', 'SERestoration', 'UNCCD', 'WorldResources', 'andyheald', 'NaturalEngland', 'BiovIntCIAT_eng', 'GlobalLF', 'ICRAF', 'FAOForestry', 'USDA', 'CGIAR', 'UNEP', 'FAO', 'IUCN', 'giz_gmbh', 'IUFRO'}
{'SERestoration', 'JAppliedEcology', 'UNCCD', 'EuanRitchie1', 'UNEP', 'Johndliu', 'GlobalLF', 'IUCN'}
{'SERestoration', 'ESA_org', 'LeightonReid', 'mobotgarden', 'UNEP', 'RestoreCAL', 'IUCN', 'MangroveProject', 'MassEcoRestore'}
